In [1]:
from autogen import retrieve_utils

paper = retrieve_utils.extract_text_from_pdf("data/paper.pdf")
print(paper)

GraphReader: Building Graph-based Agent to Enhance
Long-Context Abilities of Large Language Models
Shilong Li∗1, Yancheng He∗1, Hangyu Guo∗1, Xingyuan Bu∗†‡1, Ge Bai1, Jie Liu2,3,
Jiaheng Liu1, Xingwei Qu4, Yangguang Li3, Wanli Ouyang2,3, Wenbo Su1, Bo Zheng1
1Alibaba Group2The Chinese University of Hong Kong
3Shanghai AI Laboratory4University of Manchester
{zhuli.lsl, buxingyuan.bxy}@taobao.com
Abstract
Long-context capabilities are essential for large
language models (LLMs) to tackle complex
and long-input tasks. Despite numerous efforts
made to optimize LLMs for long contexts, chal-
lenges persist in robustly processing long in-
puts. In this paper, we introduce GraphReader,
a graph-based agent system designed to han-
dle long texts by structuring them into a graph
and employing an agent to explore this graph
autonomously. Upon receiving a question, the
agent first undertakes a step-by-step analysis
and devises a rational plan. It then invokes a
set of predefined functions to read n

In [2]:
import os
from autogen import ConversableAgent

agent = ConversableAgent(
    "chatbot",
    llm_config={"config_list": [{"model": "gpt-4o-mini", "api_key": os.environ.get("OPENAI_API_KEY")}]},
    code_execution_config=False,  # Turn off code execution, by default it is off.
    function_map=None,  # No registered functions, by default it is None.
    human_input_mode="NEVER",  # Never ask for human input.
)

[autogen.oai.client: 09-09 23:49:22] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [3]:
key_atom_prompt = """
You are now an intelligent assistant tasked with meticulously extracting both key elements and atomic facts from a long text. 1. Key Elements: The essential nouns (e.g., characters, times, events, places, numbers), verbs (e.g., actions), and adjectives (e.g., states, feelings) that are pivotal to the text’s narrative. 2. Atomic Facts: The smallest, indivisible facts, presented as concise sentences. These include propositions, theories, existences, concepts, and implicit elements like logic, causality, event sequences, interpersonal relationships, timelines, etc.  Requirements: ##### 1. Ensure that all identified key elements are reflected within the corresponding atomic facts. 2. You should extract key elements and atomic facts comprehensively, especially those that are important and potentially query-worthy and do not leave out details. 3. Whenever applicable, replace pronouns with their specific noun counterparts (e.g., change I, He, She to actual names). 4. Ensure that the key elements and atomic facts you extract are presented in the same language as the original text (e.g., English or Chinese). 5. You should output a total of key elements and atomic facts that do not exceed 1024 tokens. 6. Your answer format for each line should be: [Serial Number], [Atomic Facts], [List of Key Elements, separated with ‘|’] #####  Example: ##### User: One day, a father and his little son ......  Assistant: 1. One day, a father and his little son were going home. | father | little son | going home 2. ...... #####  Please strictly follow the above format. Let’s begin.
"""


In [4]:
reply = agent.generate_reply(messages=[{"content": paper, "role": "system"},{"content": "Find flaws in this paper's methods.", "role": "user"}])
print(reply)

Here are several potential flaws in the methods of the paper "GraphReader: Building Graph-based Agent to Enhance Long-Context Abilities of Large Language Models":

1. **Dependence on Graph Construction**: The effectiveness of GraphReader relies heavily on the quality and completeness of the graph constructed from the text. If key elements or atomic facts are missed during the extraction, the subsequent exploration and reasoning may lead to incomplete or incorrect answers. The method describes normalization and linking processes, but these may not always guarantee that the most relevant information is represented in the graph.

2. **Chunking Limitations**: The authors divide long texts into chunks while preserving paragraph structures. However, this approach may not always capture the necessary context for complex multi-hop questions. Key information might be divided across adjacent chunks, leading to incomplete understanding or loss of critical relationships between facts. Additionally

In [4]:
chunks = retrieve_utils.split_text_to_chunks(paper, 300, "multi_lines")

chunk_dict = {i: chunk for i, chunk in enumerate(chunks)}


max_tokens is too small to fit a single line of text. Breaking this line:
	GraphReader: Building Graph-based Agent to Enhance ...
Failed to split docs with must_break_at_empty_line being True, set to False.


In [5]:
reply = agent.generate_reply(
    messages=[
        {"content": key_atom_prompt, "role": "system"},
        {"content": chunks[0], "role": "user"},
    ]
)
print(reply)

1. Long-context capabilities are essential for large language models (LLMs) to tackle complex and long-input tasks. | Long-context capabilities | large language models | complex | long-input tasks  
2. Numerous efforts have been made to optimize LLMs for long contexts, yet challenges persist in robustly processing long inputs. | Numerous efforts | optimize | LLMs | long contexts | challenges | robustly processing | long inputs  
3. This paper introduces GraphReader, a graph-based agent system designed to handle long texts. | GraphReader | graph-based agent system | handle | long texts  
4. GraphReader structures long texts into a graph and employs an agent to explore this graph autonomously. | GraphReader | structures | long texts | graph | agent | explore | autonomously  
5. Upon receiving a question, the agent first undertakes a step-by-step analysis and devises a rational plan. | agent | receiving | question | undertakes | step-by-step analysis | devises | rational plan  
6. The age